In [ ]:
#for j in jumps:
#print(j)
j=35

# Eligible_ clusters 1 of a minimum size 
sizes1 = pd.read_csv(os.path.relpath("run_spercolation_outputs/cluster_sizes/clust_size_p"+str(j)+'.txt'))
clusters_1 = sizes1[sizes1.cluster_size > 2] 

# Eligible_ nodes 1 in eligible clusters
jump1 = pd.read_csv(os.path.relpath("run_spercolation_outputs/membTables/membership_p"+str(j)+'.txt'))
nodes_1 = pd.merge(jump1, clusters_1, on='cluster_id') 
#print(len(jump1), len(nodes0))




# Eligible_ clusters 2 of a minimum size 
sizes2 = pd.read_csv(os.path.relpath("run_spercolation_outputs/cluster_sizes/clust_size_p"+str(j)+'.txt'))
clusters_2 = sizes2[sizes2.cluster_size > 2] # eligible clusters

# Eligible_ nodes 2 in eligible clusters
jump2 = pd.read_csv(os.path.relpath("run_spercolation_outputs/membTables/membership_p"+str(j+1)+'.txt'))
nodes_2 = pd.merge(jump2, clusters_2, on='cluster_id') 
#print(len(jump2), len(nodes_2))

# Retained nodes and clusters in 1 that link to 2
nodes1 = pd.merge(nodes_1, nodes_2, on='node_id', suffixes=('_1', '_2')) # confirmed nodes and clusters




# Eligible_ clusters 3 of a minimum size 
sizes3 = pd.read_csv(os.path.relpath("run_spercolation_outputs/cluster_sizes/clust_size_p"+str(j)+'.txt'))
clusters_3 = sizes3[sizes3.cluster_size > 2]

# Eligible_ nodes 3 in eligible clusters
jump3 = pd.read_csv(os.path.relpath("run_spercolation_outputs/membTables/membership_p"+str(j+1)+'.txt'))
nodes_3 = pd.merge(jump3, clusters_3, on='cluster_id') 
#print(len(jump2), len(nodes_2))

# Retained nodes and clusters in 2 that link to 3
nodes2 = pd.merge(nodes_2, nodes_3, on='node_id', suffixes=('_2', '_3')) # confirmed nodes and clusters


In [ ]:
for j in jumps:
    level = jumps.index(j)
    sizes = pd.read_csv(os.path.relpath("run_spercolation_outputs/cluster_sizes/clust_size_p"+str(j)+'.txt'))
    jump = pd.read_csv(os.path.relpath("run_spercolation_outputs/membTables/membership_p"+str(j)+'.txt')) 
    clusters_ = sizes[sizes.cluster_size > 2] 
    nodes_ = pd.merge(jump, clusters_, on='cluster_id').add_suffix(level) 
    #print(nodes_.columns)
    
    # first jump (tree roots, smallest cluster sizes)
    if level == 0:
        forest = nodes_.copy()
        #forest[Level] = level
        edgelist = pd.DataFrame(columns=['Child','Parent', 'Parent_Level'])
        sizelist = pd.DataFrame(columns=['Cluster','Size','Level'])
    
    else:
        # in the forest, branches that don't make it to the apex die
        forest = pd.merge(forest, nodes_, left_on = forest.columns[0], right_on = nodes_.columns[0])
            
            
        # Levels:   1       2         3           4    
        # links:   01,       12,       23          34
        # columns: 1 4,      4 7,      7, 10     10, 14
        child = 1+3*(level-1) # 1, 4, 7
        parent = 4+3*(level-1) # 4, 7, 10
        childsize = 2+3*(level-1) # 2, 5, 8
        parentsize = 5+3*(level-1) # 5, 8, 11
        
        # Prepare an edgelist from forest
        unpruned = forest[[  forest.columns[child], forest.columns[parent], forest.columns[childsize], forest.columns[parentsize]  ]]
        pruned = unpruned[unpruned.duplicated() == False ]
        pruned.columns = ['Child', 'Parent', 'Childsize', 'Parentsize']
        pruned['Parent_Level'] = level
        edgelist = pd.concat([edgelist, pruned[['Child', 'Parent', 'Parent_Level']]])
        
        # Prepare an sizelist from forest
        sizelistchild = pruned[['Child','Childsize']]
        sizelistchild['ChildLevel'] = level - 1
        sizelistchild.columns = ['Cluster','Size','Level']
        
        sizelistparent = pruned[['Parent','Parentsize']]
        sizelistparent['ParentLevel'] = level
        sizelistparent.columns = ['Cluster','Size','Level']
        
        sizelistall = pd.concat([sizelistchild, sizelistparent])
        sizelistconcise = sizelistall[sizelistall.duplicated() == False]
        sizelist = pd.concat([sizelist, sizelistconcise])

In [ ]:
edf = pd.DataFrame(elist)
edf = edf[0].str.split('{', expand=True)
edf[[1,2]] = edf[1].str.split(',', n=1, expand=True)
edf[[-2,2]] = edf[2].str.split('length', expand=True)
edf = edf[[0,1,2]]
edf[['i','j']] = edf[0].str.split(' ', n=1, expand=True)
edf[[1,'osmid']] = edf[1].str.split(' ', expand = True)
edf[[2,3]] = edf[2].str.split(' ', expand = True)
edf[['w',3]] = edf[3].str.split('}', expand = True)
edf1 = edf[['i','j','w']]
edf2 = edf[['i','j','osmid']]
edf1.set_index('i', inplace = True)
edf1.to_csv(os.path.relpath("run_spercolation_inputs/londonijw.csv"))
edf2.head()